<h1> SACSD - Strategies affecting COVID-19 Spread and Deaths </h1>

<h2> Hunting for insights that could help to understand COVID-19 and ongoing clinical trials - Working on the Alternative Hypothesis</h2>

# What is included in this Notebook?

This notebook speaks on the alternate hypothesis of the TASK2 of BCG - COVID-19 AI Challenge. The main things included in this notebook for analyses are:

* **PART 1:** Detail study on how the number of social mitigation steps taken by the government reduced the cases of covid-19. Special emphasis on China's mitigation policies and its control measures of COVID-19. 

* **PART 2:** Understanding the Mobility data of People during COVID-19 and how it has affected mortality rates of COVID-19 Cases across countries.

* **PART 3:** Building an Epidemiological Model to assess the impact of social intervention and goverment strategies to contain COVID-19.

Feel free to connect - https://www.linkedin.com/in/amankumar01/

# General Overview

<H3> Task Details </H3>

The initiative is prompted by the suggestion that there may be a link between reduced rates of infection and lower case fatality rates associated with COVID-19 in countries that recommend BCG vaccine for all as opposed to countries that recommend BCG only for specific high-risk groups. We hope that the analysis done as part of this task might help discover useful information about the BCG - COVID-19 clinical trials. For example, some insights that may come from this analysis is whether factors such as the strain of BCG, the age at which people have been vaccinated, revaccination, or how long ago people have been vaccinated are important.

<h3> Key Questions for Consideration </h3>

* Is BCG vaccination causally related to reduced COVID‐19 mortality or other factors like lockdown and average age of the population are responsible for the different mortality rates?

* Do government mitigation and strategies adopted for COVID-19 like Lockdowns, Travel Restrictions helped in reduction of COVID-19 Cases and overall mortalilty rates?


# Understanding the Datasets

1. We read the Novel-Corona-Virus-2019-dataset managed by SRK into this notebook. The dataset hold s information about the cumulative case counts of COVID-19 Across the world. The dataset can be viewed and download from [here](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)

2. A dataset for COVID-19 Cases in China created by me is uploaded for this notebook.

3. The UNCOVER Dataset as a part of UNCOVER COVID-19 Challenge is also loaded into this notebook.

4. COVID19 Containment and Mitigation Measures Dataset uploaded by Paul Mooney. [See here](//https://www.kaggle.com/paultimothymooney/covid19-containment-and-mitigation-measures)

5. China Geo-JSON Document uploaded by sauravmishra1710.

6. Hackathon Dataset, for the BCG COVID-19 AI Challenge.

7. Countries ISO-CODES : To Geotag conutries with specific set of alphabetical codes. Done to merge the datasets together.

# PART 1 - Understanding the Social Mitigation Steps - China

<h3> Implementation of the Problem </h3>

The General Question which we require to study in this notebook is how the implementation of existing strategies affecting the rate of COVID-19 Infection. For the sake of this notebook we proceed with the following analogies ans steps.

1. Checking which countries are somewhat successful in controlling the rate of COVID-19 Spreads.
2. Figuring out the various measures adopted by that very countries to understand how did it affected spreads.
3. Checking the countries that displayed a much higher COVID-19 Infections and the growth is exponential as of now.
4. Figuring out where the country lacked in implementation tools.

Finally we can compare how the country that controlled the COVID-19 Proceeded with the community measures than that to the countries that currently exhibit a near to exponential growth of the COVID-19 Infections.

<h3> Importing the Essential Libraries for the notebook</h3>

In [ ]:
#Importing Libraries for data manipulation and loading files.
import re
import pandas as pd                              
import numpy as np       
import json
import datetime
import sqlite3 as sql
from scipy.integrate import odeint

#Importing libraries for graphical analyses.
import matplotlib.pyplot as plt                  
import plotly.express as px                      
import plotly.offline as py                     
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns                            

#Other essential libraries to import.
import glob                             
import os     
from urllib.request import urlopen
import warnings
warnings.filterwarnings('ignore')

#Required Libraries for analyses
!pip install pivottablejs
from pivottablejs import pivot_ui
import matplotlib.dates as mdates
from datetime import datetime, timedelta
!pip install lmfit
import lmfit

<h3> Analysis of COVID-19 Confirmed Cases and Deaths for multiple countries </h3>

In [ ]:
#Reading the cumulative cases dataset
covid_cases = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')

#Viewing the dataset
covid_cases.head()

#Grouping the coutries together for further analyses
country_list = covid_cases['Country/Region'].unique()

country_grouped_covid = covid_cases[0:1]

for country in country_list:
    test_data = covid_cases['Country/Region'] == country   
    test_data = covid_cases[test_data]
    country_grouped_covid = pd.concat([country_grouped_covid, test_data], axis=0)
    
country_grouped_covid.reset_index(drop=True)
country_grouped_covid.head()

<h3> Plotting a Running Map for observing the spread of COVID-19 Confirmed Cases </h3>

<iframe src='https://flo.uri.sh/visualisation/2025509/embed' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/2025509/?utm_source=embed&utm_campaign=visualisation/2025509' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'></a></div>

In [ ]:
#Plotting a bar graph for confirmed cases vs deaths due to COVID-19 in World.

unique_dates = country_grouped_covid['ObservationDate'].unique()
confirmed_cases = []
recovered = []
deaths = []

for date in unique_dates:
    date_wise = country_grouped_covid['ObservationDate'] == date  
    test_data = country_grouped_covid[date_wise]
    
    confirmed_cases.append(test_data['Confirmed'].sum())
    deaths.append(test_data['Deaths'].sum())
    recovered.append(test_data['Recovered'].sum())
    
#Converting the lists to a pandas dataframe.

country_dataset = {'Date' : unique_dates, 'Confirmed' : confirmed_cases, 'Recovered' : recovered, 'Deaths' : deaths}
country_dataset = pd.DataFrame(country_dataset)

#Plotting the Graph of Cases vs Deaths Globally.

fig = go.Figure()
fig.add_trace(go.Bar(x=country_dataset['Date'], y=country_dataset['Confirmed'], name='Confirmed Cases of COVID-19', marker_color='rgb(55, 83, 109)'))
fig.add_trace(go.Bar(x=country_dataset['Date'],y=country_dataset['Deaths'],name='Total Deaths because of COVID-19',marker_color='rgb(26, 118, 255)'))

fig.update_layout(title='Confirmed Cases and Deaths from COVID-19',xaxis_tickfont_size=14,
                  yaxis=dict(title='Reported Numbers',titlefont_size=16,tickfont_size=14,),
    legend=dict(x=0,y=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'),
    barmode='group',bargap=0.15, bargroupgap=0.1)
fig.show()


fig = go.Figure()
fig.add_trace(go.Bar(x=country_dataset['Date'], y=country_dataset['Confirmed'], name='Confirmed Cases of COVID-19', marker_color='rgb(55, 83, 109)'))
fig.add_trace(go.Bar(x=country_dataset['Date'],y=country_dataset['Recovered'],name='Total Recoveries because of COVID-19',marker_color='rgb(26, 118, 255)'))

fig.update_layout(title='Confirmed Cases and Recoveries from COVID-19',xaxis_tickfont_size=14,
                  yaxis=dict(title='Reported Numbers',titlefont_size=16,tickfont_size=14,),
    legend=dict(x=0,y=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'),
    barmode='group',bargap=0.15, bargroupgap=0.1)
fig.show()

<h3> General Observations from the above Running Graph </h3>

1. The cases of COVID-19 starts from China as the epicenter with first initial COVID-19 Cases reported in Australia, US, Canada.

2. Gradually cases in China increases and the confirmed cases is more than anywhere else in the world.

3. Europe emerges later as the new epicenter for the virus, where there is a rapid rise in COVID-19 Cases in European Countries. This outbreak occurs where the confirmed number of COVID-19 Cases in China saturates.

4. The confirmed cases of COVID-19 gradually spreads throughout the world, with spike in confirmed cases seen in European regions and US.

5. As of April 21st 2020, USA has the highest number of confimed COVID-19 Cases reported, with some European Countries emerging as the 2nd-4th highest cases of COVID-19

<h3> Digging down further for analyses </h3>

We observe COVID-19 Confirmed Cases saturation for countries like China, South Korea and Japan. Hence we look much deep into the dataset of Confrimed cases for these countries to understand it in a better context. Also various data measures available from the UNCOVER dataset are taken into consideration.

<h3> Analysis with the China COVID-19 Cases </h3>

In [ ]:
#Reading the dataset
search_data_china = country_grouped_covid['Country/Region'] == 'Mainland China'       
china_data = country_grouped_covid[search_data_china]

#Viewing the dataset
china_data.head()

In [ ]:
with open('../input/china-geo-json/china_geojson.json') as json_file:
    china = json.load(json_file)
    

#GroupingBy the dataset for the map

formated_gdf = china_data.groupby(['ObservationDate', 'Province/State'])['Confirmed', 'Deaths', 'Recovered'].max()
formated_gdf = formated_gdf.reset_index()
formated_gdf['Date'] = pd.to_datetime(formated_gdf['ObservationDate'])
formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')

formated_gdf['log_ConfirmedCases'] = np.log(formated_gdf.Confirmed + 1)


<h3> Analogies </h3>

As observed in the Hubei province a much similar trends are seen over the Last week of Feb - Early March during the period of which the cases stabilizes. We again confirm with plotting of all the provinces in China which I'd already done as saved as image for this dataset.

<h3> Plotting data for China Provinces </h3>

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(15,5))
img=mpimg.imread('../input/china-covid19-data/Anhui.png')
imgplot = plt.imshow(img)
plt.show()

plt.figure(figsize=(15,5))
img=mpimg.imread('../input/china-covid19-data/Beijing.png')
imgplot = plt.imshow(img)
plt.show()

for i in range(1,17):
    plt.figure(figsize=(15,5))
    img=mpimg.imread('../input/china-covid19-data/Screenshot ({}).png'.format(303+i))
    imgplot = plt.imshow(img)
    plt.show()

<h3> Deciphering the pattern </h3>

1. We observe past 22-02-2012 to 28-02-2020, cases in China stabilizes, post which not new much cases are reported. 
2. The pattern goes on same for all the provinces of China.

<h3> Hunting down for the Possible Causes </h3>

1. We look through the entire section of data available under uncover section to search for the events that occured in China during the period.
2. I'll update this section regualrly with new analyses to get the best possible reasons out for this saturation curve. 

<h3> Analysis of Stragegical Factors that Might have contributed to the case stabilization </h3>

We take help of the the pivotui tool to create a filterable drage and drop dataset to customize the views for mitigation measures taken across world. The tool's install command is uploaded into the libraries section of this notebook.



In [ ]:
#Importing the dataset
mitigation_policies = pd.read_csv("/kaggle/input/uncover/UNCOVER/HDE/acaps-covid-19-government-measures-dataset.csv")

#Generating the pivoting toolkit
pivot_ui(mitigation_policies)

<h3> Using the tool generated above. </h3>

1. We could select values - (say, country/region add drag it to the 2nd column box 
2. The value measure is dragged to the 2nd column to generate country wise view of the mitigation measures taken.

This tool is highly customizable hence feel free to fork the notebook and incorporate the tool in your analyses.

<h3> Analysis of Social Measures adopted in China </h3>

From the above analyses we observe the following trends in China

<iframe src='https://flo.uri.sh/visualisation/2038617/embed' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/2038617/?utm_source=embed&utm_campaign=visualisation/2038617' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

In [ ]:
#Analysis for all the important keywords from the dataset

plt.figure(figsize=(15,20))
frame1 = plt.gca()


img=mpimg.imread('../input/china-covid19-data/Word Art.png')
frame1.axes.get_xaxis().set_visible(False)
frame1.axes.get_yaxis().set_visible(False)
imgplot = plt.imshow(img)
plt.show()

<h3> Understanding the Chinese Government Action Plan </h3>

In [ ]:
#We load one more dataset avaialable on Mitigation measures to analyze the cases vs mitigation  measures adopted.
mitigation_measures_tot = pd.read_csv('../input/covid19-containment-and-mitigation-measures/COVID 19 Containment measures data.csv')

#Generating the pivoting toolkit
pivot_ui(mitigation_measures_tot)

# What was the amount of measures taken by China during the initial days of COVID-19?

<h3> 1. Government Rapid Action Implementation </h3>

The first case of Covid-19 was reported in 18th December 2020. Though the disease wasn't clear at the moment and it was spelled as "Viral Pneumonia", complusory isolation and confirmed case isolations were adopted and this measure forced by the government was implemented in Hubei province in Wuhan. Domestic Travel restrictions were sooner placed.

During the initial stages, lockdown imposed. State of emergency was declared in China on January 20th. Further health care systems were rapidly boosted to ensure the curve of COVID-19 stays flat are hospitals arent under stress.

<h3> 2. Did the mitigation measures in China helped to reduce the COVID-19 Cases which might have been higher? </h3>

To look forward with this we analyze the day of adoption of a mitigation measure in China vs. the actual case count. For this task the above dataset is concatenated for the daily cases report for that day and further analysis is done over the resultant dataset.

<iframe src='https://flo.uri.sh/visualisation/2053680/embed' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/2053680/?utm_source=embed&utm_campaign=visualisation/2053680' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

<h3> Analysis from the graph above </h3>

We plot the number of social measures and mitigation policies adopted across China and plotted them against the growth rate of new confirmed cases in China. Hubei (the outbreak) location in China was excluded from the analyses as it might saturate the cases. I guess the social measures are taken as to prevent the newer infection across newer locations so that there is no communal spread of the virus. Hence we exclude the outbreak location and plot the graph.

1. We observe the Growth of cases in China (excluding Hubei) on Feb 1, 2020 was 20.42%
2. Post this mark the level and growth rate of cases declined.
3. A high amount of social measures were taken across the China during this period. 
4. This might have further contributed to decline of COVID-19 virus to reach newer locations.

<h3> What the world saw in mitigation measures? </h3>

We take help of various open source charts and analyses available on Statista as the data for analyses of the trends are insufficient to draw conclusions from it. The following are the details that we would look over.

<iframe src='https://flo.uri.sh/visualisation/2048271/embed' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/2048271/?utm_source=embed&utm_campaign=visualisation/2048271' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>


<h3> The extent till which lockdown restricted people movement across Eurpoean Unioun </h3>

<iframe src='https://flo.uri.sh/visualisation/2049461/embed' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/2049461/?utm_source=embed&utm_campaign=visualisation/2049461' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div> 

<h3> Understanding Mitigation Measures Across Europe </h3>

<h4> Reduction in citizen movement across places during COVID-19 lockdown </h4>

<img src="https://www.statista.com/graphic/1/1106086/european-city-movements-during-coronavirus-outbreak.jpg" alt="Statistic: Percentage of people moving in selected European cities in the week ending March 22, 2020 compared to typical period prior to coronavirus outbreak* | Statista" style="width: 100%; height: auto !important; max-width:1000px;-ms-interpolation-mode: bicubic;"/></a>



In the following graph obtained above, we see still notice people movement in the cities that today has the highest COVID-19 Cases across Europe, even if lockdown and restrictions were imposed on these cities. Under the PART 2 of this analyses, I would try to look ahead with how these factors affects COVID-19 Mortalities.

# PART 2 - How Mobility Factors affects COVID-19 Mortality?

<h3> About Google's COVID-19 Mobility Data </h3>

"The data show how visits and length of stayat different places change compared to a baseline. The calculation of these changes is done using thesame kind of aggregated and anonymised dataused to show popular times for places in Google Maps. Changes for each day are compared to a baseline value for that day of the week:

● The baseline is the median value, for the corresponding day of the week, during the fiveweek period 3 Jan – 6 Feb 2020.  
● The reports show trends over several weeks." - Sources : https://www.google.com/covid19/mobility/

<h3> What the data contains? </h3>

The data contains the details about the increase/reduction of Mobility by people across regions. The categories on which this is measured are:

* **Retail and Recreation** : Mobility trends for places such as restaurants, cafés, shopping centres,theme parks, museums, libraries and cinemas.
* **Supermarket and Pharmacy** : trends for places such as supermarkets, food warehouses, farmers markets, specialty foodshops and pharmacies.
* **Parks** : Mobility trends for places like national parks, public beaches, marinas, dog parks, plazas and public gardens.
* **Public Transport** : Mobility trends for places that are public transport hubs, such as underground, bus and train stations.
* **Workplaces** : Mobility trends for places of work.
* **Residential** : Mobility trends for places of residence.






<h3> Observing the Trends - Graphical Case Study : Italy </h3>

<img src="https://www.statista.com/graphic/1/1109402/mobility-trend-in-retail-and-recreation-during-coronavirus-italy.jpg" alt="Statistic: Percentage change in visits to retail and recreation facilities by mobile users during coronavirus (COVID-19) in Italy in selected months of 2020, by region | Statista" style="width: 100%; height: auto !important; max-width:1000px;-ms-interpolation-mode: bicubic;"/></a>

<h3> Understanding the Trends </h3>

A greater percentage of reduction in mobility trends for recreation were seen across Italy in March and April, however Post April, lesser percentage of reduction in mobility activities due to COVID-19 was observed in Italy. When we compare this with Mortality across months due to COVID-19 in Italy (in the graph below) we observe increase in Mortality rates after April in Italy (the country that had the highest mortality rate during these months, globally). In our later analyses we look forward much to understand mitigation measures vs. mortality rates.

<img src="https://www.statista.com/graphic/1/1076314/covid-19-case-fatality-rates-select-countries-worldwide.jpg" alt="Statistic: Case fatality rates of COVID-19 in select countries worldwide from February 25 to August 17, 2020 | Statista" style="width: 100%; height: auto !important; max-width:1000px;-ms-interpolation-mode: bicubic;"/>

<h3> Importing the Datasets </h3>

**ECDC Dataset** : From OurWorldinData, speaks about population demographics and COVID Cases, Mortality and Deaths across countries.  
**Google Mobility Trends Dataset** : Regarding Mobility Trends across countries.  
**Isocodes Dataset** : For Fetching the unique codes for conutries. (Used for merging datasets)

In [ ]:
#Taking the Isocodes Data into consideration

isocodes = pd.read_csv('../input/countries-iso-codes/wikipedia-iso-country-codes.csv')
isocodes.columns = isocodes.columns.str.replace(' ', '_').str.lower()
isocodes = isocodes.rename({"english_short_name_lower_case": 'country_name'}, axis=1)



#ACAPS Data Import

acaps = pd.read_csv('../input/uncover/RDSC-07-30-Update/RDSC-07-30-Update/HDE/acaps-covid-19-government-measures-dataset.csv')
acaps.columns = acaps.columns.str.lower()
acaps['date_implemented'] = pd.to_datetime(acaps['date_implemented'])
acaps = acaps.merge(isocodes, left_on='iso', right_on='alpha-3_code')



#Reading ECDC Dataset

ecdc = (pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
        .assign(date=lambda f: f['date'].pipe(pd.to_datetime)))
location_code = ecdc.groupby(['location', 'iso_code']).first().iloc[:, 0].reset_index().iloc[:,:2]



#Reading Google Mobility Trends Dataset and structuring it to be used foe analyses

def fetch_google(isocodes, location_code):
    google_mobility = pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', parse_dates=['date'])

    def safe_match(pat, text):
        match = re.match(pat, text)
        return match.groups()[0] if match else text

    google_mobility.columns = google_mobility.columns.map(lambda col: safe_match("(.*)_percent", col))
    google_mobility = (google_mobility
                .merge(isocodes, left_on='country_region_code', right_on='alpha-2_code', how='left')
                .merge(location_code, left_on='alpha-3_code', right_on='iso_code', how='left'))

    google_mobility = google_mobility.loc[lambda f: f['sub_region_1'].isna()].set_index(['iso_code', 'date']).select_dtypes(float).div(100)
    return google_mobility

google = fetch_google(isocodes, location_code)
google.reset_index(inplace=True)

<h3> Wrangling the Datasets </h3>

In [ ]:
#Taking only necessary columns from ECDC Dataset.
ecdc = ecdc[['iso_code', 'continent', 'location', 'date', 'total_cases', 'total_deaths']]
            
#Making a column to calculate mortalilty rate.
ecdc['mortality'] = ecdc['total_deaths']/ecdc['total_cases']*100

#Merging the Google Mobility Dataset and ECDC Dataset
mitigation = pd.merge(google,ecdc,how='inner',on=['iso_code','date'])

#Replacing NaN Values with 0 
mitigation['mortality'].replace(np.nan,0,inplace=True)

#Viewing the dataset
mitigation.head()

<h3> How Mortality Rates differ Across Continents? </h3>

Under this section we would know how mortality rates had varied across continents and how the rates are dependent on Mitigation Measures. (Continent Level)

In [ ]:
#Finding the mortality of all the countries mentioned in the mitigation dataset
mitigation_mortality = mitigation[mitigation['date'] == '2020-12-25']
print('In the Dataset we have {} Countries'.format(mitigation_mortality.shape[0]))

#Grouping the data continent wise
continent_wise = mitigation_mortality.groupby(by=['continent']).mean(['mortality'])
continent_wise

<h3> Graph 1 : Mortality in COVID-19 across Continents. (As of December 25th, 2020) </h3>
    

<iframe src='https://flo.uri.sh/visualisation/4816090/embed' title='Interactive or visual content' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/4816090/?utm_source=embed&utm_campaign=visualisation/4816090' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

<h3> Graph 2 : The Amount by which Mobility Factors were related to COVID-19 Mortalities. (Continent Level: Mortality Rates as of 25th December, 2020) </h3>

* We observe that tough Europe has the least mortality rate, there was very less mobility on the public places, strict mobility restrictions were seen on retail and transit however.

* South America, that has the highest mortality has all mobility restrictions and reductions unders place which are strict yet a surge in mortality were seen.

Hence its evident that we can't generalize a trend on a continent scale regarding the mobility patterns and mortality in COVID-19.
    

<iframe src='https://flo.uri.sh/visualisation/4816224/embed' title='Interactive or visual content' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/4816224/?utm_source=embed&utm_campaign=visualisation/4816224' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

# Drilling Down to Country-Level

<h3> Understanding the various mobility patterns vs the COVID-19 Mortality </h3>

Below are the set of some of Folium Charts generated, the charts can be:

* Each chart is divided on Continent Level.
* Charts can be drilled down to Country Level for countries that are found in each continent.
* Under the filter option, filter can be applied to select data in chart. For e.g. Mortality and transit_restriction can be compared with each other.

These charts provide a good method to compare the people's mobility and how that has affected the COVID-19 Mortalities. The axes are:

* The RHS Y-axis shows the % of increase or reduction in Mobility. (Values shown are /100. i.e a value of -0.6 denotes 60% reduction in mobility)
* The LHS Y-axis shows the Mortality Rate / 100. i.e. a value of 0.02 shows 2% mortality rate of COVID-19.
* Via the arrow keys, slides can be navigated to view the continent/region, and accordingly the country can be selected.


<iframe src='https://flo.uri.sh/story/701267/embed' title='Interactive or visual content' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/story/701267/?utm_source=embed&utm_campaign=story/701267' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

# Observations from Mobility vs COVID-19 Mortality

Via the use of the tool, graphically and numerically the most recent mobility data can be viewed and can be compared easily with mortality rates of COVID-19. Some of the most evident findings from this section are:

**For Retail and Recreational Mobility**

Countries least affected seem to have maintained a low mobility of 20 percent from the baseline since the beginning of february. There seems to be a steady decline and the mobility towards these places has reduced to almost nil recently. All these countries seem to follow a similar curve. The countries worst affected have high mobilities to these places from the beginning, and dont seem to reduce much. Though some cases the mobility has reduced by 50 percent, it is gradual and there still seems to be 100 percent mobility above the baseline, even in recent times. We can see that France, Italy and Spain have reduced their mobility recently by almost 200 percent, but the US still seems to have higher mobility, which explains the rapidly increasing number of cases till date.


**For Grocery and Pharmacy Mobility**

There is a low mobility in all the least affected countries right from the beginning. There is a slight peak in between, for New Zealand and australia, which might be explained by people deciding to buy essentials before the complete lockdown phase is initiated. It then remains quite low, with people probably going only if necessary. New Zealand sees a huge dip in mobility almost by 80 percent from baseline, which might be explained by the government mandating home deliveries of groceries and medicines, if required, after implementing of phase 4- complete lockdown. AMong the badly affected countries, while Spain, France and Italy, do see a slight dip in mobility, people in the US still seem to have widely varying mobilities, with most of it 50 percent above the baseline.


**Mobility in Parks and Public Places**

All of the least affected countries seem to have consistently maintained low and reduced mobility to parks, except Finland. This has a rise in mobility,probably due to people wanting to go outside, yet maintain distance from other people. This probably did not affect the cases as much due to the large number of urban parks that can still function as spaces for people to visit, while adhering to social distancing norms.The US continues to see people going to parks more and more, probably as staying at home all day may add to wanting to go outdoors. Without strict lockdown measures, large numbers of people are able to move around and this number is extremely high. The other countries see a reduction in mobility to parks.


**Transit Stations Mobility**

The countries least affected implemented travel bans almost immediately, sometimes even before the first cases were reported. They also implemented intra country transit bans unless absolutely necessary. New Zealand allows transport only after testing and making sure the person does not test positive, has had no contact with other positively tested individuals, and no recent travel history. These countries do see a reduction in transit, except in the US again. We see France, Italy and Spain have also stopped people from traveling to different places. But the US still has steady 50 percent above the baseline transit mobility.


**General Conclusions**

From all this, we can conclude, that social distancing is the key factor that determines the ability for a country to reduce the number of Covid-19 cases. In the countries least affected, there has been a rapid change in the behaviour of people with mobility reducing by 90 percent over all categories. It is important to acrue from this visualisation that countries which had initially mandated social distancing and followed through with strict implementation, have the least number of cases and a nearly flatlining growth rate of cases. The worst affected countries seem to have had late and relaxed implementation, which helps form a strong correlation to number of cases in the country. Overall, the reduction of mobility,even for essential services , has to be reduced drastically in order to contain the spread of Covid-19.

# PART 3 - Evaluation : Epidemiological Modelling

<h3> What are we looking for? </h3>

We try to estimate the rates of COVID-19 using the data of reported infected, recovered and deceased cases in each country. Additionally, we used the effective reproduction number to quantify the effectiveness of taken measures and the impact of social interventions such as social lockdown or closure of schools on the number of reported infected cases in Germany, Italy and Spain. The general steps of our analysis are:

* Building SEIR Model
* Comparing the SEIR Model with respective dates w.r.t Mobility Data to understand what has happened during that tenure.

**Special Thanks to resources under CORD-19 Challenge. The codes in this section were a result of the analyses done there by multiple contributors with slight adjustments.**

In [ ]:
# Databank related functions:

def read_dataset(connection,min_date,max_date,country,region): 
    df = pd.DataFrame()
    if (country == "all" or region == "all"):
        cq = "SELECT ROWID FROM DimCountry"
    else:              
        cq = "SELECT ROWID FROM DimCountry WHERE convert_name IN %s OR continent IN %s "%(country,region) 
    cid = pd.read_sql_query(cq,connection)
    sq = "SELECT DISTINCT source_id FROM data"
    sid = pd.read_sql_query(sq,connection)
    for n in range(cid.size):
        for m in range(sid.size):
            sql = "SELECT "\
                "dc.country_code "\
            	",dc.convert_name "\
                ",dc.continent "\
                ",dt.id "\
            	",dt.date "\
            	",CASE dt.day_of_week WHEN 1 THEN 'monday' "\
            	"					 WHEN 2 THEN 'tuesday' "\
            	"					 WHEN 3 THEN 'wednesday' "\
            	"					 WHEN 4 THEN 'thursday' "\
            	"					 WHEN 5 THEN 'friday' "\
            	"					 WHEN 6 THEN 'saturday' "\
            	"					 WHEN 7 THEN 'sunday' "\
            	"					 ELSE 'n/a' END day_of_week "\
            	",dt.cw calender_week "\
            	",d.cases "\
                ",d.deaths "\
                ",d.recovered "\
                ",d.cumulate_cases "\
                ",d.cumulate_deaths "\
                ",d.cumulate_recovered "\
                ",ds.name source "\
                "FROM DimTime dt "\
                "LEFT JOIN data d ON d.time_id = dt.id AND country_id = %i AND d.source_id = %i "\
                "LEFT JOIN DimCountry dc ON dc.ROWID = %i "\
                "LEFT JOIN DimSource ds ON ds.id = %i "\
                "WHERE dt.date BETWEEN '%s' AND '%s' "\
                "ORDER by dt.date"%(cid["rowid"][n],sid["source_id"][m],cid["rowid"][n],sid["source_id"][m],min_date,max_date)            
            tmp = pd.read_sql_query(sql,connection)
            df = pd.concat([tmp,df])    
            
    obj_columns = list(df.select_dtypes(include=['object']).columns.values)
    df[obj_columns] = df[obj_columns].replace([None], np.nan)
    return df


# Database connection
path = "../input/covid19-database"    # Path to the database
db = path + '/database.db'
conn = sql.connect(db)                # SQL connection to the database
min_date = "2020-03-20"
max_date = "2020-06-30" 
data = read_dataset(conn,min_date,max_date,('all'),()) 
data

The Dataset contains details about the cases, deaths, recoveries and their respective cumulative values on the basis of data, day-of-week. The source from which the data was extracted is also mentioned under the column name - 'Source'

<h3> Making Functions and Fitting Models </h3>


An epidemiological compartmental model andused the Least Squares Error (LSE) method to fit the model to data. 


This model consists of seven compartments of the population of a country <br>
{$S(t), E(t), I(t), Q(t), R(t), D(t), P(t)$} denoting time by $t$, with the following states:
*     $S\rightarrow Susceptible$
*     $E\rightarrow Exposed$
*     $I\rightarrow Infected$
*     $Q\rightarrow Quarantined$
*     $R\rightarrow Recovered$
*     $D\rightarrow Deceased$
*     $P\rightarrow Insusceptible$    

Using this model, we assumed that a secondary infection in the investigated timespan is so unlikely that it does not warrant consideration here. Moreover the model is implemented under the assumption that every person with a positive test result goes to quarantine.

These states are connected via six transition parameters, each representing the rate of passing over from a state to the next one.

* $\alpha\rightarrow$ Protection rate
* $\beta\rightarrow$ Infection rate
* $\gamma\rightarrow$ Inverse of the average latent time 
* $\delta\rightarrow$ The rate at which infectious people enter in quarantine
* $\lambda(t)\rightarrow$ Time-dependent coefficient for recovery rate
* $\kappa(t)\rightarrow$ Time-dependent coefficient for mortality rate

Depending on whether the recovery or mortality rate in the corresponding timespan is ascending or descending, one of the three following functions is selected to fit the changes in the parameters:

\begin{equation*}
 \lambda (t)=\qquad	
 \left\{
 \begin{array}{c1}
	\dfrac{\lambda_{0}}{1 + e^{-\lambda_{1} (t + \lambda_{2})}}  \\ \\
	\lambda_{0} + e^{-\lambda_{1} (t + \lambda_{2})} \\ \\
    \lambda_{0} e^{-\lambda_{1} (t - \lambda_{2})^{2}}
 \end{array}
	\right.
\end{equation*}
where $\lambda_{0}$, $\lambda_{1}$ and $\lambda_{2}$ are empirical coefficients. 

\begin{equation*}
 \kappa (t)=\qquad	
 \left\{
 \begin{array}{c1}
	\dfrac{\kappa_{0}}{1 + e^{-\kappa_{1} (t + \kappa_{2})}}  \\ \\
	\kappa_{0} + e^{-\kappa_{1} (t + \kappa_{2})} \\ \\
    \kappa_{0} e^{-\kappa_{1} (t - \kappa_{2})^{2}}
 \end{array}
	\right.
\end{equation*}
where $\kappa_{0}$, $\kappa_{1}$ and $\kappa_{2}$ are empirical coefficients. 

<H3> Priors for the transition parameters </H3>  

According to the [Robert Koch Institute](https://edoc.rki.de/handle/176904/6547.2), we initialize the transition parameters $\alpha$, $\beta$ and $\gamma$ with the values 0.05, 0.4, and 0.4. To initialize the coefficients $\lambda_{0}$, $\lambda_{1}$ and $\lambda_{2}$ such as $\kappa_{0}$, $\kappa_{1}$ and $\kappa_{2}$ we first fitted the model with data from the first ten days and then we perform the fitting for the whole dataset in ten days periods.


<H3> Mathematical Modelling </H3>

The dynamics of the above states and transition parameters are mathematically characterized by ordinary differential equations (ODE) as follows:
\begin{equation*}
 \qquad	
 \left\{
 \begin{array}{l0.5}
	\dfrac{dS(t)}{dt} = -\alpha S(t) - \beta \dfrac{S(t)I(t)}{N} \\ 
    \dfrac{dE(t)}{dt} = -\gamma E(t) + \beta \dfrac{S(t)I(t)}{N} \\ 
    \dfrac{dI(t)}{dt} = \gamma E(t) - \delta I(t) \\ 
    \dfrac{dQ(t)}{dt} = \delta I(t) - \lambda (t) Q(t) - \kappa (t) Q(t) \\ 
    \dfrac{dR(t)}{dt} = \lambda (t) Q(t) \\ 
    \dfrac{dD(t)}{dt} = \kappa (t) Q(t) \\ 
    \dfrac{dP(t)}{dt} = \alpha S(t) \\ 
 \end{array}
       \right.
\end{equation*}
where $N$ corresponds to the population of the considered country and it can be expressed as $S + E + I + Q + R + D + P = N$


<H3> Effective Reproduction Number </H3>  

The effective reproduction number $R(t) = \beta\delta^{-1}S(t)/N$ is a strong tool to evaluate the intensity of interventions required to control the spread of the virus. Generally, $R(t)>1$ corresponds to $\dfrac{dI(t)}{dt} + \dfrac{dE(t)}{dt} >1$, which can be interpreted as the epidemical spread of disease. Similarly, $R(t)<1$ corresponds to controlling the spread of the disease.

In [ ]:
#Loading the datetime module again

from datetime import datetime

# Functions of equations

def getlambda(tspan, Q, R, initial_guess):
    """
    Parameters
    ----------
    tspan : numpy.ndarray
        time vector [days,].
    Q : numpy.ndarray
        time series of quarantiened cases [days,].
    R : numpy.ndarray
        time series of recovered cases [days,].
    initial_guess: list
        list of initial guess for the parameters lambda0, lambda1, lambda2
        
    Returns
    -------
    lambda0 : float
        estimated value of lambda0.
    lambda1 : float
        estimated value of lambda1.
    lambda2 : float
        estimated value of lambda2.
    lambdafun : function
        estimation function for lambdafun(lambda0, lambda1, lambda2, time_span)

    """
    if np.max(R)<20:                    # Assumption: Recovered cases less than 20 can either increase or decrease
        lambdafun = lambda l0,l1,l2,t: l0 *np.exp(-(l1 *(t - l2)) **2)       # Hill-shaped sigmoide fuction
        rate = (np.diff(R)/np.median(np.diff(tspan[:])))/Q[1:]
        x = tspan[1:]
        if initial_guess == []:  
            lambda0 = np.max(rate)
            lambda1 = 0.5
            lambda2 = len(tspan)/2
        else:
            lambda0, lambda1, lambda2 = initial_guess
        params = lmfit.Parameters()
        if np.max(rate)< 1e-10:   # To prevent dividing by zero
            params.add('lambda0',lambda0,min=0,max=1)
            print('\nWarning! Recovery rate is too low for Lambda estimation. Poor estimation is expected!\n')
        else:
            params.add('lambda0',lambda0,min=0,max=np.max(rate))
        params.add('lambda1',lambda1,min=0,max=1)
        params.add('lambda2',lambda2,min=0,max=100)
        res1 = lambda params, rate, x: lambdafun(params['lambda0'].value,params['lambda1'].value,params['lambda2'].value,x) - rate
        
        fit1 = lmfit.minimize(res1,params,args=(rate,x),method='least_squares')
        lambda0 = fit1.params['lambda0'].value
        lambda1 = fit1.params['lambda1'].value
        lambda2 = fit1.params['lambda2'].value
        # lmfit.report_fit(fit1)

    else: 
        myfun1 = lambda l0,l1,l2,t: l0/(1+np.exp(-l1*(t-l2)))            # Ascending sigmoide function
        myfun2 = lambda l0,l1,l2,t: l0 + np.exp(-l1 *(t + l2))           # Descending sigmoide function
        myfun3 = lambda l0,l1,l2,t: l0 *np.exp(-(l1 *(t - l2)) **2)      # Hill-shaped sigmoide function
    
        rate = (np.diff(R)/np.median(np.diff(tspan[:])))/Q[1:]
        x = tspan[1:]
          
        if initial_guess == []:  
            lambda0 = np.max(rate)
            lambda1 = 0.5
            lambda2 = len(tspan)/2
        else:
            lambda0, lambda1, lambda2 = initial_guess
            
        params = lmfit.Parameters()
        if np.max(rate)< 1e-10:   # To prevent dividing by zero
            params.add('lambda0',lambda0,min=0,max=1)
            print('\nWarning! Recovery rate is too low for Lambda estimation. Poor estimation is expected!\n')
        else:
            params.add('lambda0',lambda0,min=0,max=np.max(rate))
  
        params.add('lambda1',lambda1,min=0,max=1)
        params.add('lambda2',lambda2,min=0,max=100)
        res1 = lambda params, rate, x: myfun1(params['lambda0'].value,params['lambda1'].value,params['lambda2'].value,x) - rate
        res2 = lambda params, rate, x: myfun2(params['lambda0'].value,params['lambda1'].value,params['lambda2'].value,x) - rate
        res3 = lambda params, rate, x: myfun3(params['lambda0'].value,params['lambda1'].value,params['lambda2'].value,x) - rate
        
        fit1 = lmfit.minimize(res1,params,args=(rate,x),method='least_squares')
        lf10 = fit1.params['lambda0'].value
        lf11 = fit1.params['lambda1'].value
        lf12 = fit1.params['lambda2'].value
        lamda1 = myfun1(lf10,lf11,lf12,x)
        dlambda1 = abs(lamda1[0]-lamda1[-1])
        # lmfit.report_fit(fit1)
        
        fit2 = lmfit.minimize(res2,params,args=(rate,x),method='least_squares')
        lf20 = fit2.params['lambda0'].value
        lf21 = fit2.params['lambda1'].value
        lf22 = fit2.params['lambda2'].value
        lamda2 = myfun2(lf20,lf21,lf22,x)
        dlambda2 = abs(lamda2[0]-lamda2[-1])
        # lmfit.report_fit(fit2)
        
        fit3 = lmfit.minimize(res3,params,args=(rate,x),method='least_squares')
        lf30 = fit3.params['lambda0'].value
        lf31 = fit3.params['lambda1'].value
        lf32 = fit3.params['lambda2'].value
        lamda3 = myfun3(lf30,lf31,lf32,x)
        dlambda3 = abs(lamda3[0]-lamda3[-1])
        # lmfit.report_fit(fit2)
        
        if  (dlambda1>dlambda2) and (dlambda1>dlambda2):
            lambdaGuess = [lf10,lf11,lf12]
            lambdafun = myfun1

        elif (dlambda2>dlambda1) and (dlambda2>dlambda3): 
            lambdaGuess = [lf20,lf21,lf22]
            lambdafun = myfun2

        else:
            lambdaGuess = [lf30,lf31,lf32]
            lambdafun = myfun3

        lambda0, lambda1, lambda2 = lambdaGuess
    return lambda0, lambda1, lambda2, lambdafun


def getkappa(tspan, Q, D, initial_guess):
    """
    Parameters
    ----------
    tspan : numpy.ndarray
        time vector [days,].
    D : numpy.ndarray
        time series of death cases [days,].
    R : numpy.ndarray
        time series of recovered cases [days,].
    initial_guess: list
        list of initial guess for the parameters kappa0, kappa1, kappa2
    Returns
    -------
    lambda0 : float
        estimated value of lambda0.
    lambda1 : float
        estimated value of lambda1.
    lambda2 : float
        estimated value of lambda2.
    lambdafun : function
        estimation function for lambdafun(lambda0, lambda1, lambda2, time_span)

    """
    if np.max(D)<10:      # Assumption: Deceased cases less than 10 can either increase or decrease
        kappafun = lambda k0,k1,k2,t: k0 *np.exp(-(k1 *(t - k2)) **2)   # Hill-shaped sigmoide function
        rate = (np.diff(D)/np.median(np.diff(tspan[:])))/Q[1:]
        x = tspan[1:]
        if initial_guess == []:
            kappa0 = np.max(rate)
            kappa1 = 0.5
            kappa2 = len(tspan)/2 
        else:
            kappa0, kappa1, kappa2 = initial_guess
        params = lmfit.Parameters()
        if np.max(rate)< 1e-10:   # To prevent dividing by zero
            params.add('kappa0',kappa0,min=0,max=1)
            print('\nWarning! Mortality rate is too low for Kappa estimation. Poor estimation is expected!\n')
        else:
            params.add('kappa0',kappa0,min=0,max=np.max(rate))
  
        params.add('kappa0',kappa0,min=0,max=np.max(rate))
        params.add('kappa1',kappa1,min=0,max=1)
        params.add('kappa2',kappa2,min=0,max=len(tspan))

        res1 = lambda params, rate, x: kappafun(params['kappa0'].value,params['kappa1'].value,params['kappa2'].value,x) - rate
        
        fit1 = lmfit.minimize(res1,params,args=(rate,x),method='least_squares')
        kappa0 = fit1.params['kappa0'].value
        kappa1 = fit1.params['kappa1'].value
        kappa2 = fit1.params['kappa2'].value
        # lmfit.report_fit(fit1)


    else:
        myfun1 = lambda k0,k1,k2,t: k0 /(1 + np.exp(-k1 *(t - k2)))            # Ascending sigmoide function
        myfun2 = lambda k0,k1,k2,t: k0 + np.exp(-k1 *(t + k2))                 # Descending sigmoide function
        myfun3 = lambda k0,k1,k2,t: k0 *np.exp(-(k1 *(t - k2)) **2)            # Hill-shaped sigmoide function
        rate = (np.diff(D)/np.median(np.diff(tspan[:])))/Q[1:]
        x = tspan[1:]
        if initial_guess == []:
            kappa0 = np.max(rate)
            kappa1 = 0.5
            kappa2 = 2 
        else:
            kappa0, kappa1, kappa2 = initial_guess        
        params = lmfit.Parameters()
        if np.max(rate)< 1e-10:   # To prevent dividing by zero
            params.add('kappa0',kappa0,min=0,max=1)
            print('\nWarning! Mortality rate is too low for Kappa estimation. Poor estimation is expected!\n')
        else:
            params.add('kappa0',kappa0,min=0,max=np.max(rate))
        params.add('kappa1',kappa1,min=0,max=1)
        params.add('kappa2',kappa2,min=0,max=len(tspan))
        res1 = lambda params, rate, x: myfun1(params['kappa0'].value,params['kappa1'].value,params['kappa2'].value,x) - rate
        res2 = lambda params, rate, x: myfun2(params['kappa0'].value,params['kappa1'].value,params['kappa2'].value,x) - rate
        res3 = lambda params, rate, x: myfun3(params['kappa0'].value,params['kappa1'].value,params['kappa2'].value,x) - rate

        fit1 = lmfit.minimize(res1,params,args=(rate,x),method='least_squares')
        kf10 = fit1.params['kappa0'].value
        kf11 = fit1.params['kappa1'].value
        kf12 = fit1.params['kappa2'].value
        kappa1 = myfun1(kf10,kf11,kf12,x)
        dkappa1 = abs(kappa1[0]-kappa1[-1])
        # lmfit.report_fit(fit1)
        
        fit2 = lmfit.minimize(res2,params,args=(rate,x),method='least_squares')
        kf20 = fit2.params['kappa0'].value
        kf21 = fit2.params['kappa1'].value
        kf22 = fit2.params['kappa2'].value
        kappa2 = myfun2(kf20,kf21,kf22,x)
        dkappa2 = abs(kappa2[0]-kappa2[-1])
        # lmfit.report_fit(fit2)
        
        fit3 = lmfit.minimize(res3,params,args=(rate,x),method='least_squares')
        kf30 = fit3.params['kappa0'].value
        kf31 = fit3.params['kappa1'].value
        kf32 = fit3.params['kappa2'].value
        kappa3 = myfun2(kf30,kf31,kf32,x)
        dkappa3 = abs(kappa3[0]-kappa3[-1])
        # lmfit.report_fit(fit2)

        if  dkappa1>dkappa2 and (dkappa1>dkappa3):
            kappaGuess = [kf10,kf11,kf12]
            kappafun = myfun1
            # lmfit.report_fit(fit1)

        elif (dkappa2>dkappa1) and (dkappa2>dkappa3): 
            kappaGuess = [kf20,kf21,kf22]
            kappafun = myfun2
            # report = lmfit.report_fit(fit2)

        else:
            kappaGuess = [kf30,kf31,kf32]
            kappafun = myfun3
            # report = lmfit.report_fit(fit3)
            
        kappa0, kappa1, kappa2 = kappaGuess
    return kappa0, kappa1, kappa2, kappafun 

def ode_model(conditions, time_span, alpha, beta, gamma, delta, lamda, kappa):
    # For the range of Lambda
    S, E, I, Q , R, D, P = conditions
    N = S + E + I + Q + R + D + P
    dSdt = -alpha*S - beta*S*I/N
    dEdt = beta*S*I/N - gamma*E
    dIdt = gamma*E - delta*I
    dQdt = delta*I - lamda*Q - kappa*Q
    dRdt = lamda*Q
    dDdt = kappa*Q
    dPdt = alpha*S
    
    dSdt = np.clip(dSdt, -N, N)
    dEdt = np.clip(dEdt, -N, N)
    dIdt = np.clip(dIdt, -N, N)
    dQdt = np.clip(dQdt, -N, N)
    dRdt = np.clip(dRdt, -N, N)
    dDdt = np.clip(dDdt, -N, N)
    dPdt = np.clip(dPdt, -N, N)
    return [dSdt, dEdt, dIdt, dQdt, dRdt, dDdt, dPdt]


def SEIQRDP(initial_conditions, parameter_guess, lambdafun, kappafun, tspan):
    """
    Parameters
    ----------
    initial_conditions : list
        list of initial conditions and population [6,]
        initial_conditions = [initE, initI, initQ, initR, initD, initN]
    parameters_guess: lmfit.parameter.Parameters
        initial guess of transition parameters
    lambdafun: function
        estimation function for lambdafun(lambda0, lambda1, lambda2, time_span)
    kappafun: function
        estimation function for lambdafun(kappa0, kappa1, kappa2, time_span)
    tspan : numpy.ndarray
        time vector [days,].

    Returns
    -------
    ode results
    """
    alpha = parameter_guess['alpha'].value
    beta = parameter_guess['beta'].value
    gamma = parameter_guess['gamma'].value
    delta = parameter_guess['delta'].value
    
    lambda0 = parameter_guess['lambda0'].value
    lambda1 = parameter_guess['lambda1'].value
    lambda2 = parameter_guess['lambda2'].value
    
    kappa0 = parameter_guess['kappa0'].value
    kappa1 = parameter_guess['kappa1'].value
    kappa2 = parameter_guess['kappa2'].value

    lamda = lambdafun(lambda0, lambda1, lambda2, tspan)
    kappa = kappafun(kappa0, kappa1, kappa2, tspan)

    model = initial_conditions
    # ODE is solved using time-varying Lambda and Kappa parameters
    for ii in range(1,len(lamda)):
        t = [tspan[ii-1],tspan[ii]]
        sol = odeint(ode_model, initial_conditions,
                   t,
                   args=(alpha, beta, gamma, delta, lamda[ii-1], kappa[ii-1])) 
        initial_conditions = [sol[1][0],sol[1][1],sol[1][2],sol[1][3],sol[1][4],sol[1][5],sol[1][6]]
        model = np.vstack((model,sol[1]))
    return model

def residual(parameter_guess, initial_conditions, QRD, lambdafun, kappafun, tspan):
    sol = SEIQRDP(initial_conditions, parameter_guess, lambdafun, kappafun, tspan)
    QRD_sim = np.vstack((sol[:,3],sol[:,4], sol[:,5]))
    error = (QRD_sim-QRD).ravel()
    return error


def model_fitting(alpha, beta, gamma, delta, lambdaGuess, kappaGuess, Q, R, D, initS, initE, initI, initP, tspan):
    
    initial_conditions = [initS,initE, initI, Q[0], R[0], D[0], initP]
    QRD = np.vstack((Q,R,D))

    recovery_rate = (np.diff(R)/np.median(np.diff(tspan[:])))/Q[1:]
    death_rate = (np.diff(D)/np.median(np.diff(tspan[:])))/Q[1:]

    lambda0, lambda1, lambda2, lambdafun = getlambda(tspan, Q, R, lambdaGuess) 
    kappa0, kappa1, kappa2, kappafun = getkappa(tspan, Q, D, kappaGuess)
    
    params = lmfit.Parameters()
    params.add('alpha', value=alpha, vary=False, min=0, max=1)
    params.add('beta', value=beta, vary=True, min=0, max=1)
    params.add('gamma', value=gamma,  vary=False, min=0, max=1)
    params.add('delta', value=delta, vary=True, min=0, max=1)
    if max(recovery_rate)<1e-10:
        params.add('lambda0', value=lambda0, vary=True, min=0, max=1)    
    else:
        params.add('lambda0', value=lambda0, vary=True, min=0, max=np.max(recovery_rate))
    params.add('lambda1', value=lambda1, vary=True, min=0, max=1)
    params.add('lambda2', value=lambda2,vary=True, min=0, max=len(tspan))
    
    if np.max(death_rate) == np.min(death_rate):
        params.add('kappa0', value=kappa0, vary=True, min=0, max=1)
    else:
        params.add('kappa0', value=kappa0, vary=True, min=0, max=np.max(death_rate))
    params.add('kappa1', value=kappa1, vary=True, min=0, max=1)
    params.add('kappa2', value=kappa2,vary=True, min=0, max=len(tspan))
    
    fit = lmfit.minimize(residual,params,args=(initial_conditions, QRD, lambdafun, kappafun, tspan),
                          method='least_squares')

    
    return fit, lambdafun, kappafun

<H3> Fitting the Data and Predictive Tasks </H3>  


For this task, we used data from cumulative cases. Under assumption that every person with a positive test result goes to quarantine, we fitted our model to data in ten-day periods. The data from the last ten days is used to evaluate the fitting results and to predict the trend of changes in states and parameters in the subsequent days.

To this aim, we implemented the function `data_fit_prediction(min_date, max_date, country, region, N, conn)` with the follwing in- and outputs:
-    In:
    - min_date; Begin date of the timespan (string, format: yyyy-mm-dd)
    - max_date; End date of the timespan (string, format: yyyy-mm-dd)
    - country; Name of selected countries (tuple of strings)
    - region; Name of selected region(s) (tuple of strings)
    - N; population of selected countries (intiger)
    - conn; SQL connection
-    Out:
    - S_f; Trend of changes in susceptible cases using fitted parameters
    - I_f; Trend of changes in infected cases using fitted parameters
    - Q; Quarantined cases from reported data
    - Q_f; Trend of changes in quarantined cases using fitted parameters
    - R; Recovered cases from reported data
    - R_f; Trend of changes in recovered cases using fitted parameters
    - D; Deceased cases from reported data
    - D_f; Trend of changes in deceased cases using fitted parameters
    - model_rest; List of predicted changes in model conditions
    - beta; Changes in infection rate
    - gamma; Inverse of latent time (fixed parameter)
    - delta; Changes in infected to quarantined rate
    - lambda; Recovery rate
    - kappa; Mortality rate
    - tspan_fit; Timespan of fitting
    - tspan_pred; Timespan of prediction
    - num_days; Total number of investigated timespan
    - rest; mod(num_days,10)

Knowing that the initial values of transition parameters may vary in different countries, we initialized the varying parameters such as, infection rate $\beta$, infected to quarantined rate $\delta$, recovery rate $\lambda$, and mortality rate $\kappa$, with a preliminary fitting with the data of the first ten days. 

In [ ]:
def data_fit_prediction(min_date,max_date, country, region, N,  conn):
    # Use query to get dataframe - Measure    
    # Database query 
    if np.size(country) == 1:
        countries = (country,'')
    else:
        countries = country
    
    if region == '':
        region = ()
    data = read_dataset(conn,min_date,max_date,countries,region)   
    # cw_data = data.groupby(['calender_week']).mean()
    day_data =data.groupby(data.index).mean()
    
    days = 10    # Data is analyzed in 10 days periods
    rest = np.mod(len(day_data), days)
    train_data = day_data[:-(10+rest)]
    tspan_fit = len(train_data.index) - np.mod(len(train_data),days)
    pred_data = day_data[-(10+rest):]
    
    # Assumption: Any person with a positive test result is in quarantine until their state changes.
    Q = day_data['cumulate_cases'].values - day_data['cumulate_recovered'].values - day_data['cumulate_deaths'].values
    R = day_data['cumulate_recovered'].values
    D = day_data['cumulate_deaths'].values
    I = day_data['cumulate_cases'].values
    
    # Memory reserve / Parameter initialization
    beta = np.zeros(int(tspan_fit/days)+1)
    beta[0] = 0.4
    delta = np.zeros(int(tspan_fit/days)+1)
    alpha = 0.05
    gamma = 0.4
    delta[0] = 0.04
    lamda = []
    kappa = []
    
    S_f = []
    I_f = []
    Q_f = []
    D_f = []
    R_f = []

    jj = 1
    
    # Initial states in the corresponding country
    Q_begin = Q[:days]
    R_begin = R[:days]
    D_begin = D[:days]
    initI = gamma*I[0] 
    initE = (1-gamma)*I[0]
    initP = alpha*N
    initS = N - (initE + initI + Q_begin[0] + R_begin[0] + D_begin[0] + initP)
    
    # Initial values of parameters may vary in different countries. 
    # The aim of following fitting is to prevent wrong initialization of infection rate, infected to quarantined rate, recovery rate and mortality rate
    tspan = np.linspace(0,days,days)
    fit, lambdafun, kappafun = model_fitting(alpha, beta[0], gamma, delta[0],[],[], Q_begin, R_begin, D_begin, initS, initE, initI, initP, tspan)
    beta[0] = fit.params['beta'].value
    delta[0] = fit.params['delta'].value
    lambdaGuess = [fit.params['lambda0'].value,fit.params['lambda1'].value,fit.params['lambda2'].value]
    kappaGuess = [fit.params['kappa0'].value,fit.params['kappa1'].value,fit.params['kappa2'].value]

    # Fitting in ten-day periods
    for ii in range(0,tspan_fit,days):
        tspan = np.linspace(0,days-1,days)
        Q_fit = Q[ii:days+ii]
        R_fit = R[ii:days+ii]
        D_fit = D[ii:days+ii]
        
        fit, lambdafun, kappafun = model_fitting(alpha, beta[jj-1], gamma, delta[jj-1],lambdaGuess, kappaGuess, Q_fit, R_fit, D_fit, initS, initE, initI, initP, tspan)
        #lmfit.report_fit(fit)
        
        # To prevent underfitting and adjust to sudden changes
        if fit.params['beta'].value < 1e-7:
            beta[jj] = beta[jj-1]
        else:
            beta[jj] = fit.params['beta'].value
        
        delta[jj] = fit.params['delta'].value
        lambdaGuess = fit.params['lambda0'].value,fit.params['lambda1'].value,fit.params['lambda2'].value
        lambda_tmp = days * [np.mean(lambdafun(lambdaGuess[0],lambdaGuess[1],lambdaGuess[2],tspan))]
        lamda= np.hstack((lamda,lambda_tmp))
        kappaGuess = fit.params['kappa0'].value,fit.params['kappa1'].value,fit.params['kappa2'].value
        kappa_tmp = days * [np.mean(kappafun(kappaGuess[0],kappaGuess[1],kappaGuess[2],tspan))]
        kappa = np.hstack((kappa, kappa_tmp))
        
        # Updating the initial conditions
        initial_conditions = [initS,initE, initI, Q_fit[0], R_fit[0], D_fit[0], initP]
        model = SEIQRDP(initial_conditions, fit.params, lambdafun, kappafun, tspan)
        S_f = np.hstack((S_f, model[:,0]))
        I_f = np.hstack((I_f, model[:,2]))
        Q_f = np.hstack((Q_f, model[:,3]))
        R_f = np.hstack((R_f,model[:,4]))
        D_f = np.hstack((D_f,model[:,5]))
        jj +=1
        initS = model[:,0][-1]
        initE = model[:,1][-1]
        initI = model[:,2][-1]
        initP = model[:,6][-1]
        
    del jj   
    

    
    
    # Time dimension
    num_days = len(day_data.index)
    tspan_pred = np.linspace(0,len(pred_data), len(pred_data)+1)
    # Prediction of rest + 10 days with the fitted parameters
    initial_conditions = [initS,initE, initI, Q_f[-1], R_f[-1], D_f[-1], initP]
    model_rest = SEIQRDP(initial_conditions,fit.params,lambdafun,kappafun,tspan_pred)
    S_f = np.hstack((S_f,model_rest[:,0][:-1]))    
    I_f = np.hstack((I_f,model_rest[:,2][:-1]))
    
    return S_f, I_f, Q, Q_f, R, R_f, D, D_f, model_rest, beta, gamma, delta, lamda, kappa, tspan_fit, tspan_pred, num_days, rest

<H3> Understanding the Case of Germany </H3> 

We would first plot our model, understand the reproduction rate of new cases and then compare it with mobility measures that were adopted during this tenure in Germany.

In [ ]:
#Building the Functions to select and query - Germany from the dataset

country = 'Germany'
region = ''
N = 83.02e06           # Population of Germany
min_date = "2020-03-20"
max_date = "2020-06-30" 

S_f, I_f, Q, Q_f, R, R_f, D, D_f, model_rest, beta, gamma,  delta, lamda, kappa, tspan_fit, tspan_pred, num_days, rest = data_fit_prediction(min_date,max_date, country, region, N,  conn)

first_day = datetime.strptime(min_date, '%Y-%m-%d')
date_list = [first_day + timedelta(days=x) for x in range(0, num_days)]
d_locator = mdates.DayLocator() 
fmt = mdates.DateFormatter('%b-%d')
plt.style.use('fivethirtyeight') 

In [ ]:
# Plotting the data

Q_f_rest = model_rest[:,3]
R_f_rest = model_rest[:,4]
D_f_rest = model_rest[:,5]

fig,ax = plt.subplots(1,1,figsize=(12,8)) 

ax.plot(date_list,Q,'d',color='dimgrey',label='Data')
ax.plot(date_list[:tspan_fit],Q_f,color='darkorange', lw=3,label='Quarantined cases')
ax.plot(date_list[tspan_fit-1:],Q_f_rest,'--',color='darkorange',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*Q_f,1.125*Q_f,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.875*Q_f_rest,1.125*Q_f_rest,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.93*Q_f_rest,1.07*Q_f_rest,alpha=0.4,color='orange') 

ax.plot(date_list,I_f,color='darkmagenta', lw=3,label='Infected cases')

ax.plot(date_list,R,'d',color='dimgrey')
ax.plot(date_list[:tspan_fit],R_f,color='green', lw=3,label='Recovered cases')
ax.plot(date_list[tspan_fit-1:],R_f_rest,'--',color='green',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*R_f,1.125*R_f,alpha=0.3,color='limegreen') 
ax.fill_between(date_list[tspan_fit-1:],0.875*R_f_rest,1.125*R_f_rest,alpha=0.3,color='limegreen') 
ax.fill_between(date_list[tspan_fit-1:],0.93*R_f_rest,1.07*R_f_rest,alpha=0.4,color='limegreen') 

ax.plot(date_list,D,'d',color='dimgrey')
ax.plot(date_list[:tspan_fit],D_f,color='red', lw=3,label='Fetalities')
ax.plot(date_list[tspan_fit-1:],D_f_rest,'--',color='red',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*D_f,1.125*D_f,alpha=0.3,color='salmon') 
ax.fill_between(date_list[tspan_fit-1:],0.875*D_f_rest,1.125*D_f_rest,alpha=0.3,color='salmon') 
ax.fill_between(date_list[tspan_fit-1:],0.93*D_f_rest,1.07*D_f_rest,alpha=0.4,color='salmon') 

ax.set_title('Cases in '+country, fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.autoscale(enable=True, axis='x', tight=True)
ax.legend()
ax.xaxis.set_minor_locator(d_locator)
ax.xaxis.set_major_formatter(fmt)
fig.autofmt_xdate()

In [ ]:
#Understanding the Reproduction Number from the model

s_fit = np.zeros((int(num_days/10)+1,))
jj = 0
for ii in range(0,len(S_f),10):
    s_fit[jj] = np.mean(S_f[ii:ii+10])   
    jj += 1
del jj
# Calculating the reproduction number
Rt = beta*s_fit[:-1]/(delta*N)

# I&Q vs R(t)
fig, axes = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [3,7]})

ax = axes[1]
ax.plot(date_list[:num_days],Q,'d',color='dimgrey',label='Data')
ax.plot(date_list[:tspan_fit],Q_f,color='darkorange', lw=3,label='Quarantined cases')
ax.plot(date_list[tspan_fit-1:],Q_f_rest,'--',color='darkorange',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*Q_f,1.125*Q_f,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.875*Q_f_rest,1.125*Q_f_rest,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.93*Q_f_rest,1.07*Q_f_rest,alpha=0.4,color='orange') 
ax.set_title('Quarantined cases vs. Infected cases in '+country, fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.plot(date_list,I_f,color='darkmagenta', lw=3,label='Infected cases')
ax.autoscale(enable=True, axis='x', tight=True)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
ax.xaxis.set_minor_locator(d_locator)
ax.xaxis.set_major_formatter(fmt)

ax = axes[0]
ax.plot(date_list[0:tspan_fit:9], Rt)
ax.fill_between(date_list[0:tspan_fit:9],0.93*Rt,1.07*Rt,alpha=0.4,color='skyblue') 
ax.plot(date_list[0:tspan_fit:9], len(date_list[0:tspan_fit:9])*[1],'--', color='royalblue', lw=1)
ax.set_title(r'Effective reproduction number in '+country+' - $R(t)$', fontsize=16)
ax.autoscale(enable=True, axis='x', tight=True)
ax.xaxis.set_minor_locator(d_locator)
ax.xaxis.set_major_formatter(fmt)

fig.autofmt_xdate()

<h3> Looking in terms of Mobility Data - Germany </h3>

<iframe src='https://flo.uri.sh/visualisation/4820480/embed' title='Interactive or visual content' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/4820480/?utm_source=embed&utm_campaign=visualisation/4820480' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>

If we look more closely at the changes of infected and quarantined cases between March 20 and June 30, and compare them with the trend of changes in the effective reproduction number, we can draw two conclusions:
* The increasing number of the infected cases in the beginning of April is controlled and begins to fall due to the social lockdown, and the reproduction number reaches $R(t)<1$
* As in the beginning of May the number of infected cases begins to gradually increase, the obligatory social distancing and partial lockdown in Germany, which has been taken until 15. May, controls this rising trend. The latter can also be seen in the changes of effective reproduction number as its value decreases and remains below one.

<H3> Understanding the Case of Italy </H3> 

Italy was amongst one of the most affected countries in Europe that highlighted a very high mortality rate of COVID. Using similar as did in Germany, we would first plot our model, understand the reproduction rate of new cases and then compare it with mobility measures that were adopted during this tenure in Italy.

In [ ]:
#Building the Functions to select and query - Italy from the dataset

country = 'Italy'
region = ''
N = 60.36e06            # Population of Italy
min_date = "2020-03-20"
max_date = "2020-06-30" 

S_f, I_f, Q, Q_f, R, R_f, D, D_f, model_rest, beta, gamma,  delta, lamda, kappa, tspan_fit, tspan_pred, num_days, rest = data_fit_prediction(min_date,max_date, country, region, N,  conn)

first_day = datetime.strptime(min_date, '%Y-%m-%d')
date_list = [first_day + timedelta(days=x) for x in range(0, num_days)]
d_locator = mdates.DayLocator() 
fmt = mdates.DateFormatter('%b-%d')
plt.style.use('fivethirtyeight') 

In [ ]:
#Plotting the data

Q_f_rest = model_rest[:,3]
R_f_rest = model_rest[:,4]
D_f_rest = model_rest[:,5]

fig,ax = plt.subplots(1,1,figsize=(12,8)) 

ax.plot(date_list,Q,'d',color='dimgrey',label='Data')
ax.plot(date_list[:tspan_fit],Q_f,color='darkorange', lw=3,label='Quarantined cases')
ax.plot(date_list[tspan_fit-1:],Q_f_rest,'--',color='darkorange',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*Q_f,1.125*Q_f,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.875*Q_f_rest,1.125*Q_f_rest,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.93*Q_f_rest,1.07*Q_f_rest,alpha=0.4,color='orange') 

ax.plot(date_list,I_f,color='darkmagenta', lw=3,label='Infected cases')

ax.plot(date_list,R,'d',color='dimgrey')
ax.plot(date_list[:tspan_fit],R_f,color='green', lw=3,label='Recovered cases')
ax.plot(date_list[tspan_fit-1:],R_f_rest,'--',color='green',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*R_f,1.125*R_f,alpha=0.3,color='limegreen') 
ax.fill_between(date_list[tspan_fit-1:],0.875*R_f_rest,1.125*R_f_rest,alpha=0.3,color='limegreen') 
ax.fill_between(date_list[tspan_fit-1:],0.93*R_f_rest,1.07*R_f_rest,alpha=0.4,color='limegreen') 

ax.plot(date_list,D,'d',color='dimgrey')
ax.plot(date_list[:tspan_fit],D_f,color='red', lw=3,label='Fetalities')
ax.plot(date_list[tspan_fit-1:],D_f_rest,'--',color='red',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*D_f,1.125*D_f,alpha=0.3,color='salmon') 
ax.fill_between(date_list[tspan_fit-1:],0.875*D_f_rest,1.125*D_f_rest,alpha=0.3,color='salmon') 
ax.fill_between(date_list[tspan_fit-1:],0.93*D_f_rest,1.07*D_f_rest,alpha=0.4,color='salmon') 

ax.set_title('Cases in '+country, fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.autoscale(enable=True, axis='x', tight=True)
ax.legend()
ax.xaxis.set_minor_locator(d_locator)
ax.xaxis.set_major_formatter(fmt)
fig.autofmt_xdate()

In [ ]:
#Understanding the Reproduction Number from the model

s_fit = np.zeros((int(num_days/10)+1,))
jj = 0
for ii in range(0,len(S_f),10):
    s_fit[jj] = np.mean(S_f[ii:ii+10])   
    jj += 1
del jj
# Calculating the reproduction number
Rt = beta*s_fit[:-1]/(delta*N)


#The Final Plot

fig, axes = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [3,7]})

ax = axes[1]
ax.plot(date_list[:num_days],Q,'d',color='dimgrey',label='Data')
ax.plot(date_list[:tspan_fit],Q_f,color='darkorange', lw=3,label='Quarantined cases')
ax.plot(date_list[tspan_fit-1:],Q_f_rest,'--',color='darkorange',lw=3) 
ax.fill_between(date_list[:tspan_fit],0.875*Q_f,1.125*Q_f,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.875*Q_f_rest,1.125*Q_f_rest,alpha=0.3,color='orange') 
ax.fill_between(date_list[tspan_fit-1:],0.93*Q_f_rest,1.07*Q_f_rest,alpha=0.4,color='orange') 
ax.set_title('Quarantined cases vs. Infected cases in '+country, fontsize=16)
ax.set_xlabel('Date', fontsize=14)
ax.plot(date_list,I_f,color='darkmagenta', lw=3,label='Infected cases')
ax.autoscale(enable=True, axis='x', tight=True)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
ax.xaxis.set_minor_locator(d_locator)
ax.xaxis.set_major_formatter(fmt)

ax = axes[0]
ax.plot(date_list[0:tspan_fit:9], Rt)
ax.fill_between(date_list[0:tspan_fit:9],0.93*Rt,1.07*Rt,alpha=0.4,color='skyblue') 
ax.plot(date_list[0:tspan_fit:9], len(date_list[0:tspan_fit:9])*[1],'--', color='royalblue', lw=1)
ax.set_title(r'Effective reproduction number in '+country+' - $R(t)$', fontsize=16)
ax.autoscale(enable=True, axis='x', tight=True)
ax.xaxis.set_minor_locator(d_locator)
ax.xaxis.set_major_formatter(fmt)

fig.autofmt_xdate()

<h3> Looking in terms of Mobility Data - Italy </h3>   

<iframe src='https://flo.uri.sh/visualisation/4821281/embed' title='Interactive or visual content' frameborder='0' scrolling='no' style='width:100%;height:600px;'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/4821281/?utm_source=embed&utm_campaign=visualisation/4821281' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>


Considering the changes of infected and quarantined cases in Italy and comparing them with the trend of changes of effective reproduction number in the corresponding timespan, we can see that the spreading of the virus is well controlled until about May 1, and the effective reproduction number had reached values less than one.


However, as the number of quarantined cases decrease, the effective reproduction number increases rapidly and reaches values near to one, which may indicate the instant need for further social interventions such as social distancing.

# Next Big Steps - Conclusions

This notebook talks and studies about the Government strategies, People Mobility and how it has affected the Mortality and surge of cases in COVID-19. Under the BCG - AI Challenge, working over this alternate hypothesis, I tried to answer the various questions that may arise on these very strategies, as since Govt. Policies and policymakers are of the key people that would streamline COVID-19 Vaccination delivery and help the public in Pandemic escalation.

This notebook would be updated by me to check for much newer and diverse data to analyze more trends in spread of COVID-19 and understand it through the terms of more goverment strategies and olicies. I would love to further test on more datasets across countries. Would update the notebooks with the new findings.
Contact LinkedIn - https://www.linkedin.com/in/amankumar01/

Do drop a comment if you wish to suggest something.